# Производительность объектно-реляционной модели на примере работы с Postgres

<p> https://en.wikipedia.org/wiki/PostgreSQL
<p> http://initd.org/psycopg/


```sql
CREATE TABLE "region" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(64) NOT NULL);

CREATE TABLE "group" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(256) NOT NULL);

CREATE TABLE "crimestatsocial" (
    id SERIAL PRIMARY KEY,
    data json NOT NULL,
    reg_id INTEGER NOT NULL,
    group_id INTEGER NOT NULL,
    FOREIGN KEY (reg_id)
        REFERENCES "region" (id)
        ON UPDATE CASCADE ON DELETE CASCADE,
    FOREIGN KEY (group_id)
        REFERENCES "group" (id)
        ON UPDATE CASCADE ON DELETE CASCADE);
```
```js
data = {
    "year": <number>,
    "category": <string>,
    "gender": <string>,
    "value": <number>,
}
```

In [1]:
import sys
import time

import psycopg2

sys.path.append('../config/')
from config import config

In [2]:
params = config(section="obj_postgres")
conn = psycopg2.connect(**params)
cur = conn.cursor()

In [3]:
commands = (
    """
    SELECT
        region.id AS reg_id,
        region.name AS reg_name,
        crimestatsocial.data->'year' AS year,
        "group".id AS group_id,
        "group".name AS group_name,
        crimestatsocial.data->'category' AS category,
        crimestatsocial.data->'gender' AS gender,
        crimestatsocial.data->'value' AS value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id;""",
    """
    SELECT
        region.id AS reg_id,
        region.name AS reg_name,
        crimestatsocial.data->'year' AS year,
        "group".id AS group_id,
        "group".name AS group_name,
        crimestatsocial.data->'category' AS category,
        crimestatsocial.data->'gender' AS gender,
        crimestatsocial.data->'value' AS value
    FROM crimestatsocial, region, "group"
    WHERE crimestatsocial.group_id = "group".id
    AND crimestatsocial.reg_id = region.id
    AND CAST(crimestatsocial.data->>'year' AS integer)=2016;""",
    """
    SELECT group_id, COUNT("group_id")
    FROM "crimestatsocial"
    WHERE CAST(crimestatsocial.data->>'year' AS integer)=2016
    GROUP BY "group_id";""",
    """
    SELECT crimestatsocial.data->>'year', COUNT(*)
    FROM "crimestatsocial"
    GROUP BY crimestatsocial.data->>'year';""",
    """
    SELECT COUNT("id") AS count
    FROM "group";""",
    
    """
    SELECT row_to_json(row) FROM (
        SELECT
            region.id AS reg_id,
            region.name AS reg_name,
            crimestatsocial.data->'year' AS year,
            "group".id AS group_id,
            "group".name AS group_name,
            crimestatsocial.data->'category' AS category,
            crimestatsocial.data->'gender' AS gender,
            crimestatsocial.data->'value' AS value
        FROM crimestatsocial, region, "group"
        WHERE crimestatsocial.group_id = "group".id
        AND crimestatsocial.reg_id = region.id
    ) row;""",
    """
    SELECT row_to_json(row) FROM (
        SELECT
            region.id AS reg_id,
            region.name AS reg_name,
            crimestatsocial.data->'year' AS year,
            "group".id AS group_id,
            "group".name AS group_name,
            crimestatsocial.data->'category' AS category,
            crimestatsocial.data->'gender' AS gender,
            crimestatsocial.data->'value' AS value
        FROM crimestatsocial, region, "group"
        WHERE crimestatsocial.group_id = "group".id
        AND crimestatsocial.reg_id = region.id
        AND CAST(crimestatsocial.data->>'year' AS integer)=2016
    ) row;""",
)

## 1. Собираются исходные кортежи

In [4]:
cur.execute(commands[0])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

27384
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [5]:
%timeit cur.execute(commands[0])

948 ms ± 61.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit cur.execute(commands[5])

1.37 s ± 24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. Собираются исходные кортежи за 2016 год

In [7]:
cur.execute(commands[1])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

3948
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [8]:
%timeit cur.execute(commands[1])

309 ms ± 8.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit cur.execute(commands[6])

383 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 3. Подсчет исходных кортежей за 2016 год по группам

In [10]:
cur.execute(commands[2])
cur.fetchall()

[(14002, 1128), (14007, 376), (14006, 1692), (14005, 752)]

In [11]:
%timeit cur.execute(commands[2])

181 ms ± 6.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 4. Подсчет исходных кортежей по годам

In [12]:
cur.execute(commands[3])
cur.fetchall()

[('2011', 3864),
 ('2010', 3864),
 ('2014', 3990),
 ('2015', 3990),
 ('2016', 3948),
 ('2012', 3864),
 ('2013', 3864)]

In [13]:
%timeit cur.execute(commands[3])

197 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 5. Подсчет уникальных групп

In [14]:
def fun():
    cur.execute(commands[4])
    fetchall = cur.fetchall()
    return fetchall[0][0]

fun()

4

In [15]:
%timeit fun()

793 µs ± 67.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
cur.close()
conn.close()